In [35]:
using Pkg
Pkg.add("Unicode")

 Resolving package versions...
  Updating `C:\Users\cheny\.julia\environments\v1.3\Project.toml`
 [no changes]
  Updating `C:\Users\cheny\.julia\environments\v1.3\Manifest.toml`
 [no changes]


## Using Packages

In [37]:
using .Threads
using BenchmarkTools
using SparseArrays
using LinearAlgebra

## Including External Files|

In [38]:
include("global_curved_multithreading.jl")

plot_blocks (generic function with 1 method)

## Setting SBP operator Orders

In [39]:
SBPp = 6

6

## Reading .inp files and initializing boundary conditions

In [41]:
bc_map = [BC_DIRICHLET, BC_DIRICHLET, BC_NEUMANN, BC_NEUMANN, BC_JUMP_INTERFACE]
# 1 refers to Dirichlet boundary condition
# 2 refers to Neumann boundary condition
# 7 refers to Jump interface condition

5-element Array{Int64,1}:
 1
 1
 2
 2
 7

In [42]:
(verts, EToV, EToF, FToB, EToDomain) = read_inp_2d("../meshes/4_4_block.inp", bc_map=bc_map)

([0.5 -0.5 … 0.25 0.25; 0.5 0.5 … -0.5 -0.25], [4 7 … 9 6; 23 24 … 16 21; 24 20 … 20 16; 25 25 … 22 22], [1 5 … 9 29; 2 4 … 39 36; 3 6 … 30 40; 4 7 … 38 39], [1, 0, 1, 0, 0, 1, 0, 0, 0, 0  …  0, 1, 1, 0, 1, 0, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [48]:
verts'

25×2 Array{Float64,2}:
  0.5    0.5 
 -0.5    0.5 
 -0.5   -0.5 
  0.5   -0.5 
  0.0    0.5 
 -0.5    0.0 
  0.0   -0.5 
  0.5    0.0 
  0.0    0.0 
  0.25   0.5 
  0.0    0.25
  0.25   0.0 
  0.5    0.25
  0.25   0.25
 -0.5    0.25
 -0.25   0.0 
 -0.25   0.5 
 -0.25   0.25
 -0.25  -0.5 
  0.0   -0.25
 -0.5   -0.25
 -0.25  -0.25
  0.5   -0.25
  0.25  -0.5 
  0.25  -0.25

In [44]:
EToV

4×16 Array{Int64,2}:
  4   7   9   8   1   5   9   8   2   6   9   5   3   7   9   6
 23  24  20  12  10  11  12  13  15  16  11  17  19  20  16  21
 24  20  12  23  13  10  11  12  17  15  16  11  21  19  20  16
 25  25  25  25  14  14  14  14  18  18  18  18  22  22  22  22

In [47]:
FToB

40-element Array{Int64,1}:
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 1
 0
 1
 ⋮
 0
 0
 0
 1
 1
 0
 1
 0
 1
 0
 0
 1

In [49]:
EToDomain

16-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [51]:
(nelems, nfaces) = (size(EToV, 2), size(FToB, 1))
@show (nelems, nfaces)

(nelems, nfaces) = (16, 40)


(16, 40)

In [54]:
# This is needed to fix up points that should be on the boundary of the
# circle, but the mesh didn't quite put them there
for v in 1:size(verts, 2)
    x,y = verts[1,v], verts[2,v]
    if abs(hypot(x,y) - 1) < 1e-5
        Q = atan(y,x)
        verts[1,v], verts[2,v] = cos(Q), sin(Q)
    end
end


In [56]:
plot_connectivity(verts, EToV)

                                        connectivity
      ┌────────────────────────────────────────────────────────────────────────────────┐ 
    1 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
      │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

## Setting base mesh sizes in each direction

In [57]:
N1 = N0 = 16

16

In [62]:
EToN0 = zeros(Int64, 2, nelems)
EToN0[1,:] .= N0;
@show EToN0
EToN0[2,:] .= N1;
@show EToN0

EToN0 = [16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
EToN0 = [16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16; 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]


2×16 Array{Int64,2}:
 16  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16
 16  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16

In [69]:
# Checking types and sizes 
@assert typeof(EToV) == Array{Int, 2} && size(EToV) == (4, nelems)
@assert typeof(EToF) == Array{Int, 2} && size(EToF) == (4, nelems)
@assert maximum(maximum(EToF)) == nfaces   # The maximum number of EToF element should equal the number of faces 

# Determine secondary arrays
# FToE : Unique Global Face to Element Number
#        (the i'th column of this stores the element numbers that share the
#        global face number i)
# FToLF: Unique Global Face to Element local face number
#        (the i'th column of this stores the element local face numbers that
#        shares the global face number i)
# EToO : Element to Unique Global Faces Orientation
#        (the i'th column of this stores the whether the element and global
#        face are oriented in the same way in physical memory or need to be
#        rotated)
# EToS : Element to Unique Global Face Side
#        (the i'th column of this stores whether an element is on the
#        plus side or minus side of the global face)


## Forming exact solutions

In [72]:
Lx = maximum(verts[1,:])
@show Lx
Ly = maximum(abs.(verts[2,:]))
@show Ly

Lx = 0.5
Ly = 0.5


0.5

In [73]:
(kx, ky) = (2*π / Lx, 4*π / Ly)

(12.566370614359172, 25.132741228718345)

In [74]:
vex(x,y,e) = begin
    if EToDomain[e] == 1
        return cos.(kx * x) .* cosh.(ky * y)
    elseif EToDomain[e] == 2
        return 10 .+ cos.(kx * x) .* cosh.(ky * y)
    else
        error("invalid block")
    end
end

vex (generic function with 1 method)

In [ ]:
vex_x(x,y,e) = begin
    if EToDomain[e] == 1
        